In [1]:
import pandas as pd
import requests
import sys
def pull_data(url):


    headers = {
                                    "Host": "stats.nba.com",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                                    "Accept": "application/json, text/plain, */*",
                                    "Accept-Language": "en-US,en;q=0.5",
                                    "Accept-Encoding": "gzip, deflate, br",

                                    "Connection": "keep-alive",
                                    "Referer": "https://stats.nba.com/"
                                }
    json = requests.get(url,headers = headers).json()
    if len(json["resultSets"])== 1:

        
        data = json["resultSets"][0]["rowSet"]
        #print(data)
        columns = json["resultSets"][0]["headers"]
        #print(columns)
        
        df = pd.DataFrame.from_records(data, columns=columns)
    else:

        data = json["resultSets"]["rowSet"]
        print(data)
        #print(json)
        columns = json["resultSets"]["headers"][1]['columnNames']
        #print(columns)
        df = pd.DataFrame.from_records(data, columns=columns)
    return df
rs_frames=[]


for year in range(1997,2025):
    season = f"{year-1}-{str(year)[-2:]}"
    stype = 'Regular%20Season'
    url = 'https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Four%20Factors&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType='+stype+'&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
    df = pull_data(url)
    url2='https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType='+stype+'&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
    df2 = pull_data(url2)
    df=df.merge(df2)
    url3 ='https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Opponent&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
    df3 = pull_data(url3)
    df3col=['TEAM_ID', 'OPP_FTA','OPP_FGA',  'OPP_PTS']
    df3=df3[df3col]
    df3['OPP_TS_PCT'] = df3['OPP_PTS'] / (2 * (df3['OPP_FGA'] + (0.44 * df3['OPP_FTA'])))

    df=df.merge(df3,on='TEAM_ID')
    #print(df)
    #sys.exit()
    df['year']=year
    rs_frames.append(df)
    print(year)
ps_frames=[]
for year in range(1997,2025):
    season = f"{year-1}-{str(year)[-2:]}"
    stype = 'Playoffs'
    url = 'https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Four%20Factors&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType='+stype+'&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
    df = pull_data(url)
    url2='https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType='+stype+'&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
    df2 = pull_data(url2)
    df=df.merge(df2)
    url3 ='https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Opponent&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType='+stype+'&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='

    df3 = pull_data(url3)
    df3col=['TEAM_ID', 'OPP_FTA',  'OPP_FGA',  'OPP_PTS']
    df3=df3[df3col]
    df=df.merge(df3,on='TEAM_ID')
    df['year']=year
    ps_frames.append(df)
    print(year)


1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024


In [2]:
rs_ff=pd.concat(rs_frames)
ps_ff=pd.concat(ps_frames)

ps_ff[ps_ff.year==2020]

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,EFG_PCT,FTA_RATE,TM_TOV_PCT,...,AST_RATIO_RANK,DREB_PCT_RANK,REB_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK,OPP_FTA,OPP_FGA,OPP_PTS,year
0,1610612738,Boston Celtics,17,10,7,0.588,831.0,0.527,0.285,0.142,...,9,9,3,7,13,2,25.1,86.0,104.4,2020
1,1610612751,Brooklyn Nets,4,0,4,0.000,192.0,0.482,0.268,0.140,...,4,5,12,15,2,16,23.5,91.0,126.3,2020
2,1610612742,Dallas Mavericks,6,2,4,0.333,293.0,0.549,0.297,0.121,...,13,4,9,6,3,11,27.5,90.2,126.7,2020
3,1610612743,Denver Nuggets,19,9,10,0.474,917.0,0.545,0.260,0.146,...,7,13,7,5,15,8,24.6,82.2,109.7,2020
4,1610612745,Houston Rockets,12,5,7,0.417,581.0,0.535,0.235,0.134,...,11,10,15,8,7,10,22.8,83.7,106.9,2020
5,1610612754,Indiana Pacers,4,0,4,0.000,192.0,0.530,0.252,0.157,...,6,15,16,10,10,14,29.8,80.8,111.3,2020
6,1610612746,LA Clippers,13,7,6,0.538,629.0,0.544,0.291,0.130,...,12,6,6,4,8,6,23.6,84.8,109.8,2020
7,1610612747,Los Angeles Lakers,21,16,5,0.762,1008.0,0.561,0.293,0.158,...,1,7,2,2,11,1,25.3,81.4,106.0,2020
8,1610612748,Miami Heat,21,14,7,0.667,1018.0,0.537,0.322,0.135,...,2,11,8,3,12,3,23.1,84.8,108.0,2020
9,1610612749,Milwaukee Bucks,10,5,5,0.500,485.0,0.537,0.279,0.146,...,5,2,5,9,5,7,24.2,84.9,109.9,2020


In [3]:
rs_ff=pd.concat(rs_frames)
rs_ff.to_csv('teamfactors.csv',index = False)

ps_ff=pd.concat(ps_frames)
ps_ff.to_csv('teamfactors_ps.csv',index = False)

In [4]:
import pandas as pd
rs_ff = pd.read_csv('teamfactors.csv')
print(rs_ff.columns)
ps_ff = pd.read_csv('teamfactors_ps.csv')

def sum_factors(rs_ff):
    frames=[]
    for year in range(1997,2025):
        year_ff=rs_ff[rs_ff.year==year].reset_index(drop=True)
        
        year_ff['OPP_TS_PCT'] = year_ff['OPP_PTS'] / (2 * (year_ff['OPP_FGA'] + (0.44 * year_ff['OPP_FTA'])))

        columns = ['TEAM_ID', 'TEAM_NAME', 'TS_PCT','EFG_PCT',
           'FTA_RATE', 'TM_TOV_PCT', 'OREB_PCT', 'OPP_TS_PCT','OPP_EFG_PCT', 'OPP_FTA_RATE',
           'OPP_TOV_PCT', 'OPP_OREB_PCT','NET_RATING','OFF_RATING', 'DEF_RATING', 'year']
        year_ff=year_ff[columns]
        avg_rating=year_ff['OFF_RATING'].sum()/len(year_ff)
    
        year_ff['RDEF_RATING'] = year_ff['DEF_RATING']- avg_rating
        year_ff['ROFF_RATING'] = year_ff['OFF_RATING'] -  avg_rating
        frames.append(year_ff)
    total = pd.concat(frames)
    return total

rs_df = sum_factors(rs_ff)
rs_df.to_csv('fourfactors.csv')
ps_df = sum_factors(ps_ff)
ps_df.to_csv('fourfactors_ps.csv')
print(rs_df.columns)

Index(['TEAM_ID', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'EFG_PCT',
       'FTA_RATE', 'TM_TOV_PCT', 'OREB_PCT', 'OPP_EFG_PCT', 'OPP_FTA_RATE',
       'OPP_TOV_PCT', 'OPP_OREB_PCT', 'GP_RANK', 'W_RANK', 'L_RANK',
       'W_PCT_RANK', 'MIN_RANK', 'EFG_PCT_RANK', 'FTA_RATE_RANK',
       'TM_TOV_PCT_RANK', 'OREB_PCT_RANK', 'OPP_EFG_PCT_RANK',
       'OPP_FTA_RATE_RANK', 'OPP_TOV_PCT_RANK', 'OPP_OREB_PCT_RANK',
       'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TO', 'AST_RATIO',
       'DREB_PCT', 'REB_PCT', 'TS_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS',
       'PIE', 'OFF_RATING_RANK', 'DEF_RATING_RANK', 'NET_RATING_RANK',
       'AST_PCT_RANK', 'AST_TO_RANK', 'AST_RATIO_RANK', 'DREB_PCT_RANK',
       'REB_PCT_RANK', 'TS_PCT_RANK', 'PACE_RANK', 'PIE_RANK', 'OPP_FTA',
       'OPP_FGA', 'OPP_PTS', 'OPP_TS_PCT', 'year'],
      dtype='object')
Index(['TEAM_ID', 'TEAM_NAME', 'TS_PCT', 'EFG_PCT', 'FTA_RATE', 'TM_TOV_P

In [10]:
ps_df

,TEAM_ID,TEAM_NAME,TS_PCT,EFG_PCT,FTA_RATE,TM_TOV_PCT,OREB_PCT,OPP_TS_PCT,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT,NET_RATING,OFF_RATING,DEF_RATING,year,RDEF_RATING,ROFF_RATING
0,1610612737,Atlanta Hawks,0.541,0.485,0.369,0.187,0.323,0.517199,0.484,0.251,0.127,0.319,-3.4,103.6,107.0,1997,1.94375,-1.45625
1,1610612766,Charlotte Hornets,0.535,0.492,0.257,0.149,0.355,0.641357,0.629,0.235,0.165,0.252,-10.6,107.1,117.7,1997,12.64375,2.04375
2,1610612741,Chicago Bulls,0.512,0.470,0.289,0.142,0.358,0.523819,0.469,0.389,0.186,0.307,5.9,106.0,100.1,1997,-4.95625,0.94375
3,1610612765,Detroit Pistons,0.513,0.471,0.265,0.114,0.237,0.550971,0.496,0.338,0.190,0.317,-2.4,101.9,104.3,1997,-0.75625,-3.15625
4,1610612745,Houston Rockets,0.566,0.516,0.369,0.167,0.348,0.536307,0.483,0.314,0.136,0.333,2.1,111.6,109.6,1997,4.54375,6.54375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,1610612752,New York Knicks,0.558,0.521,0.278,0.121,0.368,0.599289,0.567,0.237,0.127,0.306,-2.5,118.1,120.6,2024,10.01250,7.51250
12,1610612760,Oklahoma City Thunder,0.563,0.530,0.216,0.118,0.253,0.530915,0.501,0.233,0.166,0.317,6.4,110.9,104.5,2024,-6.08750,0.31250
13,1610612753,Orlando Magic,0.527,0.481,0.299,0.151,0.304,0.531031,0.493,0.257,0.154,0.243,4.9,104.9,100.0,2024,-10.58750,-5.68750
14,1610612755,Philadelphia 76ers,0.574,0.525,0.296,0.128,0.316,0.541175,0.507,0.268,0.117,0.370,0.2,116.9,116.7,2024,6.11250,6.31250


In [5]:
for year in range(1997,2025):
    print(year)
    
    yrs_df = rs_df[rs_df.year==year]
    #print('Year Min TS')
    #print(yrs_df['TS_PCT'].min())
    print('Year Min OPP TS')
    print(yrs_df['OPP_TS_PCT'].min())
    #print('Year Max TS')
    #print(yrs_df['TS_PCT'].max())
    print('Year max OPP TS')
    print(yrs_df['OPP_TS_PCT'].max())

1997
Year Min OPP TS
0.509359410180566
Year max OPP TS
0.5751599147121536
1998
Year Min OPP TS
0.4824676174277116
Year max OPP TS
0.5616760184298161
1999
Year Min OPP TS
0.4637538326763031
Year max OPP TS
0.5516259620089862
2000
Year Min OPP TS
0.49014401631335225
Year max OPP TS
0.5519895888477148
2001
Year Min OPP TS
0.48289122929685796
Year max OPP TS
0.5458163045816304
2002
Year Min OPP TS
0.4948383710248896
Year max OPP TS
0.5407194879717502
2003
Year Min OPP TS
0.4886260983821959
Year max OPP TS
0.5376169438669438
2004
Year Min OPP TS
0.4800683371298405
Year max OPP TS
0.5402949978623343
2005
Year Min OPP TS
0.5024325921884236
Year max OPP TS
0.5647507383082823
2006
Year Min OPP TS
0.5014003071641522
Year max OPP TS
0.5730406418055188
2007
Year Min OPP TS
0.5106783349959191
Year max OPP TS
0.5684481950304735
2008
Year Min OPP TS
0.507435026254247
Year max OPP TS
0.567858860565782
2009
Year Min OPP TS
0.5085110967463908
Year max OPP TS
0.5710911980224243
2010
Year Min OPP TS
0.517

In [11]:
for year in range(1997,2025):
    print(year)
    
    yrs_df = ps_df[ps_df.year==year]
    #print('Year Min TS')
    #print(yrs_df['TS_PCT'].min())
    print('Year Min OPP TS')
    print(yrs_df['OPP_TS_PCT'].min())
    #print('Year Max TS')
    #print(yrs_df['TS_PCT'].max())
    print('Year max OPP TS')
    print(yrs_df['OPP_TS_PCT'].max())

1997
Year Min OPP TS
0.49735363758763174
Year max OPP TS
0.6413567493112947
1998
Year Min OPP TS
0.49246298190226334
Year max OPP TS
0.5725611098876899
1999
Year Min OPP TS
0.46330107723206143
Year max OPP TS
0.6185256564683209
2000
Year Min OPP TS
0.4524640209332752
Year max OPP TS
0.5497743229689067
2001
Year Min OPP TS
0.48624338624338626
Year max OPP TS
0.5640824557472552
2002
Year Min OPP TS
0.48937719523687145
Year max OPP TS
0.5798518213624203
2003
Year Min OPP TS
0.48774359870663286
Year max OPP TS
0.5642209829208783
2004
Year Min OPP TS
0.4712243658616341
Year max OPP TS
0.5864167829476014
2005
Year Min OPP TS
0.49536880392488586
Year max OPP TS
0.6113021057155136
2006
Year Min OPP TS
0.5210210886806056
Year max OPP TS
0.6123291441152188
2007
Year Min OPP TS
0.5061816878695575
Year max OPP TS
0.5724110912810692
2008
Year Min OPP TS
0.5176263984043015
Year max OPP TS
0.5776750799857803
2009
Year Min OPP TS
0.5155674194462817
Year max OPP TS
0.605938645917181
2010
Year Min OPP T

In [7]:
ps_df

,TEAM_ID,TEAM_NAME,TS_PCT,EFG_PCT,FTA_RATE,TM_TOV_PCT,OREB_PCT,OPP_TS_PCT,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT,NET_RATING,OFF_RATING,DEF_RATING,year,RDEF_RATING,ROFF_RATING
0,1610612737,Atlanta Hawks,0.541,0.485,0.369,0.187,0.323,0.517199,0.484,0.251,0.127,0.319,-3.4,103.6,107.0,1997,1.94375,-1.45625
1,1610612766,Charlotte Hornets,0.535,0.492,0.257,0.149,0.355,0.641357,0.629,0.235,0.165,0.252,-10.6,107.1,117.7,1997,12.64375,2.04375
2,1610612741,Chicago Bulls,0.512,0.470,0.289,0.142,0.358,0.523819,0.469,0.389,0.186,0.307,5.9,106.0,100.1,1997,-4.95625,0.94375
3,1610612765,Detroit Pistons,0.513,0.471,0.265,0.114,0.237,0.550971,0.496,0.338,0.190,0.317,-2.4,101.9,104.3,1997,-0.75625,-3.15625
4,1610612745,Houston Rockets,0.566,0.516,0.369,0.167,0.348,0.536307,0.483,0.314,0.136,0.333,2.1,111.6,109.6,1997,4.54375,6.54375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,1610612752,New York Knicks,0.558,0.521,0.278,0.121,0.368,0.599289,0.567,0.237,0.127,0.306,-2.5,118.1,120.6,2024,10.01250,7.51250
12,1610612760,Oklahoma City Thunder,0.563,0.530,0.216,0.118,0.253,0.530915,0.501,0.233,0.166,0.317,6.4,110.9,104.5,2024,-6.08750,0.31250
13,1610612753,Orlando Magic,0.527,0.481,0.299,0.151,0.304,0.531031,0.493,0.257,0.154,0.243,4.9,104.9,100.0,2024,-10.58750,-5.68750
14,1610612755,Philadelphia 76ers,0.574,0.525,0.296,0.128,0.316,0.541175,0.507,0.268,0.117,0.370,0.2,116.9,116.7,2024,6.11250,6.31250
